## Thông tin nhóm

**Thành viên thứ 1:**

- Họ và tên: Trần Trung Hiếu
- MSSV: 1712442

**Thành viên thứ 2:**
- Họ và tên: Lê Long Hồ
- MSSV: 1712447

**Thành viên thứ 3:**
- Họ và tên: Nguyễn Đình Thiên Phúc
- MSSV: 18120144

___

## Import

In [1]:
!pip3 install nltk
!pip3 install pyvi

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re
from pyvi import ViTokenizer
from nltk.stem.porter import PorterStemmer
import string

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_rows', 500)

  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/75/aca08032c9752a75acc60ff7f4e58f1a74164b996395f44727ffdb17ebaf/regex-2020.11.13-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/34/5b/bd0f0fb5564183884d8e35b81d06d7ec06a20d1a0c8b4c407f1554691dce/joblib-1.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/02/8f8880a4fd6625461833abcf679d4c12a44c76f9925f92bf212bb6cefaad/tqdm-4.56.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/10/e1/0e5bc6b5e3327b9385d6e0f1b0a7c0404f28b74eb6db59a778515b30fd9c/pyvi-0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/25/74/5b7befa513482e6dee1f3dd68171a6c9dfc14c0eaa00f885ffeba54fe9b0/sklearn_crfsuite-0.3.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pa

[nltk_data] Downloading package punkt to
[nltk_data]     /home/tthieu22_06/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

## Thu thập dữ liệu

TODO

---

## Khám phá dữ liệu

In [2]:
movie_df = pd.read_csv('movie_final.csv')
movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
5158,https://www.themoviedb.org/movie/432976,Icarus\n(2017),78.0,poster_final/432976.jpg,12,While investigating the furtive world of illeg...,Truth is the new banned substance,Phim Tài Liệu,NaN,en
2130,https://www.themoviedb.org/tv/82596,Emily in Paris\n(2020),81.0,poster_final/82596.jpg,TV-MA,When ambitious Chicago marketing exec Emily un...,Pardon her French.,"Phim Chính Kịch, Phim Hài",NaN,en
1073,https://www.themoviedb.org/tv/10079,Spider-Man Unlimited\n(1999),73.0,poster_final/10079.jpg,NaN,Spider-Man Unlimited was a short-lived America...,NaN,"Phim Hoạt Hình, Action & Adventure",NaN,en
4889,https://www.themoviedb.org/movie/522212,Công Lý Bất Toàn\n(2019),80.0,poster_final/522212.jpg,PG-13,"Sau khi tốt nghiệp Harvard, Bryan đến Alabama ...",Every generation has its hero. Meet ours.,Phim Chính Kịch,NaN,vi
1347,https://www.themoviedb.org/tv/114865,Strafrechters\n(2021),0.0,NaN,NaN,We don't have an overview translated in Englis...,NaN,NaN,NaN,en


In [3]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5598 non-null   object 
 1   title            5594 non-null   object 
 2   score            5594 non-null   float64
 3   posterImagePath  5326 non-null   object 
 4   certification    4177 non-null   object 
 5   overview         5594 non-null   object 
 6   tagline          2522 non-null   object 
 7   genres           5230 non-null   object 
 8   trailerUrl       79 non-null     object 
 9   language         5594 non-null   object 
dtypes: float64(1), object(9)
memory usage: 437.5+ KB


In [4]:
n_row, n_col = movie_df.shape
print("Dữ liệu có số dòng:", n_row)
print("Dữ liệu có số cột:", n_col)

Dữ liệu có số dòng: 5598
Dữ liệu có số cột: 10


Mỗi dòng trong tập dữ liệu thể hiện vài thông tin của một bộ phim, bộ phim đó có thể là phim truyền hình hoặc phim điện ảnh. Các dòng này đều có ý nghĩa tương đương nhau. 

## Đưa ra câu hỏi cần trả lời

Các thông tin được tải lên trên Internet hầu như không được kiểm duyệt, chính vì vậy sẽ có thể xuất hiện những nội dung không phù hợp với mọi lứa tuổi, cụ thể là những bộ phim có gán nhãn giới hạn độ tuổi người lớn không nên để những người chưa đủ tuổi, đặc biệt là trẻ nhỏ tiếp xúc. Tuy nhiên, rất nhiều các website xem phim lại không có gán nhãn giới hạn độ tuổi. Chính vì vậy, câu hỏi này được đưa ra nhằm tìm câu trả lời giải quyết vấn đề nội dung bộ phim không phù hợp với các lứa tuổi, giúp các phụ huynh có thể quản lí được trẻ nhỏ không tìm coi những bộ phim có nội dung không phù hợp.

Câu hỏi này sẽ liên quan tới việc phân loại nội dung tóm tắt của một bộ phim và dự đoán độ tuổi phù hợp để coi bộ phim này là gì.

- Input: overview (nội dung tóm tắt của bộ phim)
- Output: certification (độ tuổi phù hợp)

---

## Khám phá dữ liệu

In [5]:
print("Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:")
print(movie_df.language.value_counts())

Danh sách các ngôn ngữ thể hiện nội dung tóm tắt:
en    4304
vi    1224
lt      31
fi      10
tl       5
pt       4
cy       4
ro       3
es       2
af       1
cs       1
it       1
de       1
fr       1
id       1
sk       1
Name: language, dtype: int64


In [6]:
print('Kiểu dữ liệu của cột cerfitication:', movie_df.certification.dtype)

Kiểu dữ liệu của cột cerfitication: object


In [7]:
print("Số lượng dòng bị thiếu output:", movie_df.certification.isnull().sum())
print("Số lượng dòng có output:", movie_df.shape[0] - movie_df.certification.isnull().sum())

Số lượng dòng bị thiếu output: 1421
Số lượng dòng có output: 4177


In [8]:
print('Số lượng lớp trong certification:', len(movie_df.certification.unique()))
print('Tỉ lệ bộ phim của từng lớp:\n', movie_df.certification.value_counts(normalize=True)*100)

Số lượng lớp trong certification: 88
Tỉ lệ bộ phim của từng lớp:
 R             14.891070
TV-14         12.401245
PG-13         10.892985
TV-MA          9.743835
PG             9.743835
TV-PG          4.907829
G              4.141729
12             4.045966
NR             3.998085
15             3.303807
16             2.657410
TV-Y7          2.250419
TV-G           1.723725
U              1.484319
18             1.173091
14             1.125209
L              1.053388
16+            0.694278
0              0.598516
10             0.598516
6              0.550634
18+            0.478813
T              0.430931
12+            0.406991
MA15+          0.383050
保護級            0.359109
TV-Y           0.359109
普遍級            0.335169
AL             0.335169
6+             0.287288
7              0.287288
輔12級           0.239406
M              0.215466
UA             0.215466
NC-17          0.215466
APTA           0.215466
11             0.191525
M/12           0.167584
9              0.14364

## Tiền xử lí - tách các tập dữ liệu

### Lọc thông tin cần thiết

`en` và `vi` là hai ngôn ngữ có số lượng bộ phim chiếm nhiều nhất, cho nên nhóm sẽ giữ lại những bộ phim thuộc một trong hai ngôn ngữ này để thực hiện tiền xử lí và loại bỏ đi các dòng chứa các ngôn ngữ còn lại

In [9]:
envi_movie_df = movie_df[((movie_df.language == 'vi') | (movie_df.language == 'en'))]
envi_movie_df.reset_index(inplace=True, drop=True)
envi_movie_df.head(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language
0,https://www.themoviedb.org/tv/44217,Huyền Thoại Vikings\n(2013),79.0,poster_final/44217.jpg,14+,Người Viking đưa chúng ta đến thế giới tàn khố...,NaN,"Action & Adventure, Phim Chính Kịch",NaN,vi
1,https://www.themoviedb.org/tv/82856,The Mandalorian\n(2019),85.0,poster_final/82856.jpg,TV-14,"After the fall of the Galactic Empire, lawless...",Bounty hunting is a complicated profession.,"Sci-Fi & Fantasy, Action & Adventure",NaN,en
2,https://www.themoviedb.org/tv/77169,Cobra Kai\n(2018),81.0,poster_final/77169.jpg,TV-14,This Karate Kid sequel series picks up 30 year...,Cobra Kai never dies.,"Action & Adventure, Phim Chính Kịch",NaN,en
3,https://www.themoviedb.org/tv/75006,Học Viện Umbrella\n(2019),87.0,poster_final/75006.jpg,TV-MA,"Được đưa về quá khứ những năm 1960 tại Dallas,...",Super. Dysfunctional. Family.,"Action & Adventure, Sci-Fi & Fantasy, Phim Chí...",NaN,vi
4,https://www.themoviedb.org/tv/71712,Bác Sĩ Thiên Tài\n(2017),86.0,poster_final/71712.jpg,TV-14,Một bác sĩ trẻ mắc hội chứng bác học được nhận...,"His mind is a mystery, his methods are a miracle.",Phim Chính Kịch,NaN,vi


Xóa các dòng bị thiếu nhãn `certification`

In [10]:
notnull_envi_movie_df = envi_movie_df.dropna(subset=['certification']).reset_index(drop=True)
print('Số lượng bộ phim sau khi loại bỏ nhãn thiếu:', notnull_envi_movie_df.shape[0])

Số lượng bộ phim sau khi loại bỏ nhãn thiếu: 4171


Vì số lượng lớp lớn và phân bố không đồng đều, thực hiện chia các lớp giới hạn tuổi thành 2 cụm là: lớn hơn 18 (>=18) và nhỏ hơn 18 (<18)

Danh sách cụm này được lưu trong tệp `age_restricted.txt`

In [11]:
# Danh sách các nhóm tuổi
age_cert = {}
with open('resources/age_restricted.txt', 'r') as cert_file:
    for line in cert_file:
        cert, age = line.strip().split(',')
        age_cert[cert] = 1 if age == '>18' else 0

notnull_envi_movie_df['is_adult'] = notnull_envi_movie_df.certification.apply(lambda c: age_cert[c])

In [12]:
notnull_envi_movie_df.sample(5)

,url,title,score,posterImagePath,certification,overview,tagline,genres,trailerUrl,language,is_adult
1888,https://www.themoviedb.org/movie/458576,Thợ Săn Quái Vật\n(2020),57.0,poster_final/458576.jpg,PG-13,Monster Hunter được chuyển thể từ series game ...,"Behind our world, there is another","Phim Giả Tượng, Phim Hành Động, Phim Phiêu Lưu",https://www.themoviedb.org/video/play?key=puQy...,vi,0
2377,https://www.themoviedb.org/movie/468816,Diều Hâu Trên Núi\n(2019),61.0,poster_final/468816.jpg,16,"Đám người Mông Cổ, do Khan Burunda chỉ huy, di...",Freedom is my nature.,"Phim Lịch Sử, Phim Chính Kịch, Phim Hành Động",NaN,vi,0
1194,https://www.themoviedb.org/tv/76747,On My Block\n(2018),86.0,poster_final/76747.jpg,TV-14,A coming of age comedy following a diverse gro...,NaN,Phim Hài,NaN,en,0
1804,https://www.themoviedb.org/movie/592350,Học Viện Anh Hùng: Anh Hùng Trỗi Dậy\n(2019),84.0,poster_final/592350.jpg,PG,Tác giả đã giải thích rằng movie thứ hai này c...,Boku no Hero Academia the Movie 2: Heroes Rising,"Phim Hoạt Hình, Phim Hành Động, Phim Giả Tượng...",NaN,vi,0
3508,https://www.themoviedb.org/movie/20453,3 Chàng Ngốc\n(2009),80.0,poster_final/20453.jpg,12세 이상 관람가,Phim xoay quanh câu chuyện ăn học của 3 anh ch...,Don't be Stupid. Be an I.D.I.O.T.,"Phim Chính Kịch, Phim Hài",NaN,vi,0


### Tách các tập dữ liệu

Vì hai ngôn ngữ `en` và `vi` có đặc điểm khác nhau, để tránh chiều dữ liệu quá lớn và làm giảm hiệu quả của mô hình phân loại, ta sẽ chia `en` và `vi` ra làm 2 tập dữ liệu riêng biệt. Đồng thời, do chỉ tập trung vào `overview` cho nên sẽ loại bỏ các cột khác chỉ giữ lại `overview` và `is_adult`

In [13]:
en_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'en'][['overview', 'is_adult']]
vi_movie_df = notnull_envi_movie_df[notnull_envi_movie_df.language == 'vi'][['overview', 'is_adult']]

len(en_movie_df), len(vi_movie_df)

(2968, 1203)

In [14]:
# Phân bố lớp trên tập tiếng Anh
en_movie_df.is_adult.value_counts()

0    2255
1     713
Name: is_adult, dtype: int64

In [15]:
# Phân bố lớp trên tập tiếng Việt
vi_movie_df.is_adult.value_counts()

0    784
1    419
Name: is_adult, dtype: int64

Thực hiện chia thành các tập train, valid, và test với tỉ lệ lần lượt là 70/15/15

In [16]:
# Tách tập cho tiếng Việt
vi_y_sr = vi_movie_df.is_adult
vi_X_df = vi_movie_df.drop(columns=['is_adult'])

vi_train_X, vi_tmp_X, vi_train_y, vi_tmp_y = train_test_split(vi_X_df, vi_y_sr, stratify=vi_y_sr, random_state=42, test_size=0.3)
vi_valid_X, vi_test_X, vi_valid_y, vi_test_y = train_test_split(vi_tmp_X, vi_tmp_y, stratify=vi_tmp_y, random_state=42, test_size=0.15)


In [17]:
vi_train_X.shape, vi_train_y.shape

((842, 1), (842,))

In [18]:
vi_valid_X.shape, vi_valid_y.shape

((306, 1), (306,))

In [19]:
vi_test_X.shape, vi_test_y.shape

((55, 1), (55,))

In [20]:
# Tách tập cho tiếng Anh
en_y_sr = en_movie_df.is_adult
en_X_df = en_movie_df.drop(columns=['is_adult'])

en_train_X, en_tmp_X, en_train_y, en_tmp_y = train_test_split(en_X_df, en_y_sr, stratify=en_y_sr, random_state=42, test_size=0.3)
en_valid_X, en_test_X, en_valid_y, en_test_y = train_test_split(en_tmp_X, en_tmp_y, stratify=en_tmp_y, random_state=42, test_size=0.5)

In [21]:
en_train_X.shape, en_train_y.shape

((2077, 1), (2077,))

In [22]:
en_valid_X.shape, en_valid_y.shape

((445, 1), (445,))

In [23]:
en_test_X.shape, en_test_y.shape

((446, 1), (446,))

---

## Khám phá dữ liệu - tập huấn luyện

Kiểm tra xem liệu có những bộ phim nào bị trùng nội dung, điều này có thể là do bộ phim có nhiều phần hay nhiều tập riêng lẻ.

In [24]:
en_overview_dup = en_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", en_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(en_train_X[en_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 292
3 nội dung tóm tắt trùng nhiều nhất:
We don't have an overview translated in English. Help us expand our database by adding one.                                                                                                                                                                                                                73
An emotional thrill ride through the day-to-day chaos of the city's most explosive hospital and the courageous team of doctors who hold it together. They will tackle unique new cases inspired by topical events, forging fiery relationships in the pulse-pounding pandemonium of the emergency room.     2
Closet witch Teresa Palmer and centuries-old vampire Matthew Goode are drawn into a deadly mystery and forbidden romance when a magical book shows up in an Oxford library.                                                                                                                                 2
Name: overview, d

Như quan sát, có tương đối bộ phim có cùng nội dung là `We don't have an overview translated in English. Help us expand our database by adding one.`, tức là những bộ phim này không có nội dung tóm tắt. Việc đưa vào mô hình huấn luyện những nội dung này sẽ làm giảm khả năng chính xác của mô hình vì nó không đem lại ý nghĩa về mặt phân loại.

In [25]:
word_dict = Counter()
max_leng = -1
for words in en_train_X.overview.apply(lambda s: word_tokenize(s)):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(302,
 [(',', 6132),
  ('the', 5523),
  ('.', 4675),
  ('a', 3547),
  ('and', 3422),
  ('of', 3141),
  ('to', 2953),
  ('in', 1876),
  ('his', 1342),
  ('is', 1326),
  ('with', 907),
  ('her', 790),
  ('on', 785),
  ("'s", 782),
  ('an', 727),
  ('by', 721),
  ('their', 690),
  ('he', 674),
  ('for', 668),
  ('as', 666)])

In [26]:
vi_overview_dup = vi_train_X.duplicated(['overview'])
print("Số lượng nội dung tóm tắt bị trùng:", vi_overview_dup.sum())

print("3 nội dung tóm tắt trùng nhiều nhất:")
print(vi_train_X[vi_overview_dup].overview.value_counts()[:3])
    

Số lượng nội dung tóm tắt bị trùng: 120
3 nội dung tóm tắt trùng nhiều nhất:
Ở thế giới nơi đủ loại động vật cùng tồn tại, chú sói hiền lành nhận thức được bản năng săn mồi của mình khi ngôi trường của chú ứng phó với một kẻ sát nhân đang ẩn náu.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                2
Akira (Chúa tể A

Nhìn chung thì dữ liệu tiếng Việt ít dữ liệu trùng hơn, có thể không có bộ phim mà không có nội dung tóm tắt cho nên ở bước tiền xử lí không cần phải xử lí trường hợp này.

Vì đặc điểm của tiếng Việt hơi khác so với tiếng Anh. Trong tiếng Anh mỗi token sẽ ngăn cách nhau bởi những khoảng trắng.
Tuy nhiên, đối với tiếng Việt thì có chút khác biệt. Ví dụ trong tiếng Việt từ `hạnh` và `phúc` nếu đi riêng lẻ sẽ mang ý nghĩa khác với khi đi chung `hạnh_phúc`. Trong khi đó tiếng Anh chỉ là `happy`

In [27]:
word_dict = Counter()
max_leng = -1
for words in vi_train_X.overview.apply(lambda s: ViTokenizer.tokenize(s).split()):
    word_dict.update(words)
    max_leng = max(max_leng, len(words))

max_leng, word_dict.most_common(20)

(203,
 [(',', 3167),
  ('.', 2782),
  ('của', 1774),
  ('một', 1575),
  ('và', 1463),
  ('là', 969),
  ('người', 794),
  ('những', 792),
  ('trong', 664),
  ('được', 656),
  ('với', 643),
  ('đã', 607),
  ('có', 501),
  ('khi', 478),
  ('về', 452),
  ('để', 450),
  ('phim', 447),
  (')', 422),
  ('(', 421),
  ('bị', 396)])

Ở cả 2 ngôn ngữ thì số lượng những từ và kí tự chiếm nhiều nhất là dấu câu (punctuation) và stopwords, để tránh những từ và kí tự không có ý nghĩa cho phân loại bài toán này ảnh hưởng lên hiệu suất phân loại, những từ và kí tự này sẽ được loại bỏ ở bước tiền xử lí.

## Tiền xử lí

Đầu tiên sẽ loại bỏ những mẫu có cùng nội dung tóm tắt trong tiếng Anh

In [28]:
# Remove duplicate
most_duplicate = "We don't have an overview translated in English. Help us expand our database by adding one."
en_train_y = en_train_y[en_train_X.overview != most_duplicate]
en_train_X = en_train_X[en_train_X.overview != most_duplicate]

assert len(en_train_X) == len(en_train_y)

Quá trình tiền xử lí nội dung của hai ngôn ngữ khá tương đồng, bao gồm loại bỏ dấu câu, chuẩn hóa in thường.
Điều khác biệt của xử lí hai ngôn ngữ như sau:
- Đối với tiếng Anh, việc đầu tiên là sẽ tách nội dung tóm tắt thành các token bằng hàm `word_tokenize`. Sau đó, để giảm không gian từ điển thì sẽ qua một bước xử lí `stemming` để đưa những từ giống nhau về tiền tố hay nói cách khác là có chung nguồn gốc. Ví dụ: playing, plays, played sẽ được đưa về play.
- Đối với tiếng Việt, sử dụng đối tượng `ViTokenizer` để `tokenize` nội dung tóm tắt thành những token

In [29]:
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lang='en'):
        self.stemmer = PorterStemmer()    # Initialize stemming object
        self.lang = lang
        with open('resources/en_stopwords.txt', 'r') as en_file: 
            self.en_stopwords = [line.strip() for line in en_file]
        with open('resources/vi_stopwords.txt', 'r') as vi_file: 
            self.vi_stopwords = [line.strip() for line in vi_file]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None, lang=None):
        transformed_X = X.apply(lambda s: re.sub(f'[{string.punctuation}]', ' ', s.lower()))    # remove punctuation
        if lang is None:
            lang = self.lang
        if lang == 'en':
            # remove stopword, stemming and tokenize English sentences
            transformed_X = transformed_X.apply(lambda s: [self.stemmer.stem(word) for word in word_tokenize(s) 
                                                           if word not in self.en_stopwords])
        elif lang == 'vi':
            # remove stopword and tokenize Vietnamese sentences
            transformed_X = transformed_X.apply(lambda s: [word for word in ViTokenizer.tokenize(s).split() 
                                                           if word not in self.vi_stopwords])
        else:
            raise Exception('we only support `en` and `vi` language')
        
        return transformed_X

In [30]:
transformer = FeatureTransformer()
transformed_en_train_X = transformer.transform(en_train_X.overview, lang='en')
transformed_vi_train_X = transformer.transform(vi_train_X.overview, lang='vi')

## Tiền xử lí - tập validation

Thực hiện tiền xử lí trên tập valid tương tự với tập huấn luyện

In [31]:
transformed_en_valid_X = transformer.transform(en_valid_X.overview, lang='en')
transformed_vi_valid_X = transformer.transform(vi_valid_X.overview, lang='vi')

---

## Mô hình hóa dữ liệu

Xây dựng và huấn luyện mô hình ngôn ngữ. `LogisticRegression` là mô hình thích hợp cho bái toán binary classification. Vậy nên mô hình này sẽ được sử dụng cho bài toán phân loại nội dung tóm tắt này.

#### Mô hình phân loại cho tiếng Anh

In [32]:
# both preprocess and classifier
en_full_pipeline = Pipeline([
    ('preprocess', FeatureTransformer(lang='en')),
    ('vectorize', TfidfVectorizer(tokenizer=lambda text: text, lowercase=False, token_pattern=None)),
    ('clf', LogisticRegression(C=1, solver='lbfgs', random_state=42, max_iter=1000))
])

# classifier pipeline
en_clf_pipeline = Pipeline([
    ('vectorize', TfidfVectorizer(tokenizer=lambda text: text, lowercase=False, token_pattern=None)),
    ('clf', LogisticRegression(C=1, solver='lbfgs', random_state=42, max_iter=1000))
])

# Train
en_clf_pipeline.fit(transformed_en_train_X, en_train_y)

# Độ chính xác trên tập train
train_pred = en_clf_pipeline.predict(transformed_en_train_X)
train_acc = np.mean(train_pred == en_train_y)*100
print("Độ chính xác tập train:", train_acc)

# Độ chính xác trên tập valid
valid_pred = en_clf_pipeline.predict(transformed_en_valid_X)
valid_acc = np.mean(valid_pred == en_valid_y)*100
print("Độ chính xác tập valid:", valid_acc)

Độ chính xác tập train: 77.88317523714429
Độ chính xác tập valid: 77.52808988764045


#### Mô hình phân loại cho tiếng Việt

In [33]:
# both preprocess and classifier
vi_full_pipeline = Pipeline([
    ('preprocess', FeatureTransformer(lang='vi')),
    ('vectorize', TfidfVectorizer(tokenizer=lambda text: text, lowercase=False, token_pattern=None)),
    ('clf', LogisticRegression(C=1, solver='lbfgs', random_state=42, max_iter=1000))
])

# classifier pipeline
vi_clf_pipeline = Pipeline([
    ('vectorize', TfidfVectorizer(tokenizer=lambda text: text, lowercase=False, token_pattern=None)),
    ('clf', LogisticRegression(C=1, solver='lbfgs', random_state=42, max_iter=1000))
])

# Train
vi_clf_pipeline.fit(transformed_vi_train_X, vi_train_y)

# Độ chính xác trên tập train
train_pred = vi_clf_pipeline.predict(transformed_vi_train_X)
train_acc = np.mean(train_pred == vi_train_y)*100
print("Độ chính xác tập train:", train_acc)

# Độ chính xác trên tập valid
valid_pred = vi_clf_pipeline.predict(transformed_vi_valid_X)
valid_acc = np.mean(valid_pred == vi_valid_y)*100
print("Độ chính xác tập valid:", valid_acc)

Độ chính xác tập train: 84.79809976247031
Độ chính xác tập valid: 71.56862745098039


### Hyperparameter tuning

Các thông số có thể được điều chỉnh để nâng hiệu suất của mô hình bao gồm:
- `ngram_range`: là thông số của `TfidfVectorizer` thể hiện chuỗi token liên tiếp nhau. Ví dụ: đối với từ "hạnh phúc" khi ngram = 1 thì chuỗi sẽ là ['hạnh', 'phúc'], ngram = 2 thì chuỗi sẽ là ['hạnh phúc']
- `max_df`: là thông số của `TfidfVectorizer`. Nếu một token có tần suất xuất hiện vượt qua ngưỡng này thì sẽ bị loại bỏ khỏi từ điển.
- `C`: là thông số của `Logistic Regression`. Là hệ số được áp dụng để phạt giá trị của các tham số nhằm tránh overfitting.

In [34]:
# Thiết lập các siêu tham số
ngram_ranges = [(1, 1), (1, 2)]
max_ds = [0.8, 0.9, 1.0]
C_param = [0.1, 1, 5, 7, 10, 100, 1000]

#### Tìm mô hình tốt nhất cho tiếng Anh

In [35]:
en_best_val_acc = 0
en_best_C = None; 
en_best_ngram = None
en_best_maxdf = None
en_train_accs = []
en_valid_accs = []

# brute-force search
for ngram_range in ngram_ranges:
    for max_d in max_ds:
        for C in C_param:
            en_clf_pipeline.set_params(vectorize__ngram_range=ngram_range, vectorize__max_df=max_d, clf__C=C)
            en_clf_pipeline.fit(transformed_en_train_X, en_train_y)
            
            # Tính độ chính xác trên tập train
            train_pred = en_clf_pipeline.predict(transformed_en_train_X)
            train_acc = np.mean(train_pred == en_train_y) * 100
            en_train_accs.append(train_acc)
            
            # Tính độ chính xác trên tập valid
            val_pred = en_clf_pipeline.predict(transformed_en_valid_X)
            valid_acc = np.mean(val_pred == en_valid_y) * 100
            en_valid_accs.append(valid_acc)
            
            # Tìm siêu tham số tốt nhất
            if en_best_val_acc < valid_acc:
                en_best_val_acc = valid_acc
                en_best_C = C
                en_best_ngram = ngram_range
                en_best_maxdf = max_d

In [36]:
en_best_C, en_best_ngram, en_best_maxdf

(1000, (1, 2), 0.8)

#### Tìm mô hình tốt nhất cho tiếng Việt

In [37]:
vi_best_val_acc = 0
vi_best_C = None; 
vi_best_ngram = None
vi_best_maxdf = None
vi_train_accs = []
vi_valid_accs = []

# brute-force search
for ngram_range in ngram_ranges:
    for max_d in max_ds:
        for C in C_param:
            vi_clf_pipeline.set_params(vectorize__ngram_range=ngram_range, vectorize__max_df=max_d, clf__C=C)
            vi_clf_pipeline.fit(transformed_vi_train_X, vi_train_y)
            
            # Tính độ chính xác trên tập train
            train_pred = vi_clf_pipeline.predict(transformed_vi_train_X)
            train_acc = np.mean(train_pred == vi_train_y) * 100
            vi_train_accs.append(train_acc)
            
            # Tính độ chính xác trên tập valid
            val_pred = vi_clf_pipeline.predict(transformed_vi_valid_X)
            valid_acc = np.mean(val_pred == vi_valid_y) * 100
            vi_valid_accs.append(valid_acc)
            
            # Tìm siêu tham số tốt nhất
            if vi_best_val_acc < valid_acc:
                vi_best_val_acc = valid_acc
                vi_best_C = C
                vi_best_ngram = ngram_range
                vi_best_maxdf = max_d

In [38]:
vi_best_C, vi_best_ngram, vi_best_maxdf

(100, (1, 1), 0.8)

### Đánh giá mô hình

Đánh giá mô hình trên tập test.

#### Hiệu suất mô hình tiếng Anh

In [39]:
# Huấn luyện lại toàn bộ mô hình trên siêu tham số mới
en_full_pipeline.set_params(vectorize__ngram_range=en_best_ngram, vectorize__max_df=en_best_maxdf, clf__C=en_best_C)
en_full_pipeline.fit(en_train_X.overview, en_train_y)

test_pred = en_full_pipeline.predict(en_test_X.overview)
test_acc = np.mean(test_pred == en_test_y)*100
print("Test Accuracy: %.2f" % test_acc)

Test Accuracy: 83.86


#### Hiệu suất mô hình tiếng Việt

In [40]:
# Huấn luyện lại toàn bộ mô hình trên siêu tham số mới
vi_full_pipeline.set_params(vectorize__ngram_range=vi_best_ngram, vectorize__max_df=vi_best_maxdf, clf__C=vi_best_C)
vi_full_pipeline.fit(vi_train_X.overview, vi_train_y)

test_pred = vi_full_pipeline.predict(vi_test_X.overview)
test_acc = np.mean(test_pred == vi_test_y)*100
print("Test Accuracy: %.2f" % test_acc)

Test Accuracy: 65.45


---

## Nhìn lại quá trình làm đồ án

### Những khó khăn

- Giới hạn độ tuổi mà trang phim cung cấp không thống nhất, nhiều metadata khác nhau cho nên dữ liệu thu thập chứa đa dạng lớp. Để giải quyết vấn đề này, gộp các lớp lại còn 2 cụm >18 và <18 thì phải thực hiện một cách thủ công.

### Bài học rút ra

- Cách triển khai một mô hình xử lí ngôn ngữ đơn giản sử dụng Bag of word
- Đặc điểm khác nhau giữa xử lí ngôn ngữ tiếng Anh và tiếng Việt

Nếu có thêm thời gian, nhóm sẽ tận dụng những thêm những đặc trưng khác để mô hình có thể xử lí được nhiều trường hợp phức tạp hơn. Đồng thời sẽ tìm hiểu thêm những mô hình nổi bật trong xử lí ngôn ngữ tự nhiên như Word2vec, BERT,... để mô hình nâng cao hơn, có thể hiểu được ngữ nghĩa và thứ tự của các từ.

## Tài liệu tham khảo

1. [scikit-learn User Guide](https://scikit-learn.org/)
2. [Stackoverflow](https://stackoverflow.com/)
3. [Pandas Documentation](https://pandas.pydata.org/)
4. [Stemming and Lemmatization in Python](https://www.datacamp.com/community/tutorials/stemming-lemmatization-python)
5. [Vietnamese stopwords](https://github.com/stopwords/vietnamese-stopwords)